In [25]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

sns.set_theme(style="whitegrid", context="notebook")

In [26]:
# Discover and load prediction CSVs
from pathlib import Path

def find_results_dir(start: Path = Path.cwd()) -> Path:
    # Walk up until we find the repo folder that contains Prediction_Results
    for p in [start] + list(start.parents):
        cand = p / 'Prediction_Results'
        if cand.exists() and cand.is_dir():
            return cand
    raise FileNotFoundError("Could not locate 'Prediction_Results' directory by walking up from current path.")

results_dir = find_results_dir()

csv_files = sorted(results_dir.glob('_davis*.csv'))
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {results_dir}")

csv_files

FileNotFoundError: No CSV files found in /Users/cheng.wang/Documents/GitHub/llm-drug-agent/Prediction_Results

In [27]:
# Load and standardize columns: Affinity (true), Predicted (prediction), and Split
from typing import Tuple, List

ID_COLS = ['Index_ID','Drug_ID','Target_ID']


def standardize_df(df: pd.DataFrame) -> pd.DataFrame:
    # Clean up column names: strip whitespace and unify multiple spaces
    df = df.copy()
    df.columns = [str(c).strip() for c in df.columns]
    cols = {c.lower(): c for c in df.columns}
    # Prefer explicit columns (case-insensitive)
    true_col = cols.get('affinity')
    pred_col = cols.get('predicted')
    split_col = cols.get('split')
    # Fallbacks if needed
    if true_col is None:
        for c in ['true', 'y_true', 'label', 'affinity_true']:
            if c in cols:
                true_col = cols[c]
                break
    if pred_col is None:
        for c in ['y_pred', 'pred', 'prediction', 'affinity_pred']:
            if c in cols:
                pred_col = cols[c]
                break
    if split_col is None:
        for c in ['set', 'fold', 'partition']:
            if c in cols:
                split_col = cols[c]
                break
    if true_col is None or pred_col is None:
        raise ValueError(f"Expected columns 'Affinity' (true) and 'Predicted' (prediction) not found. Got: {list(df.columns)}")
    out = df.rename(columns={true_col: 'y_true', pred_col: 'y_pred'})
    if split_col and split_col in out.columns:
        out = out.rename(columns={split_col: 'Split'})
    else:
        # No Unknown category requested; use NaN when Split is missing
        out['Split'] = np.nan
    # Keep optional metadata columns if present
    keep_extra = [c for c in ID_COLS if c in out.columns]
    return out[['y_true', 'y_pred', 'Split'] + keep_extra]

# Load all files into a tidy dataframe with model name inferred from filename
frames = []
for f in csv_files:
    df = pd.read_csv(f)
    std = standardize_df(df)
    std['model'] = f.stem.replace('_predictions', '')
    frames.append(std)

all_preds = pd.concat(frames, ignore_index=True)
# Filter to allowed splits only (no Unknown)
allowed_splits = ['Train', 'Validation', 'Test']
all_preds = all_preds[all_preds['Split'].isin(allowed_splits)].copy()
# Ensure Split is categorical with a preferred order
all_preds['Split'] = pd.Categorical(all_preds['Split'], categories=allowed_splits, ordered=True)
all_preds.head()

ValueError: No objects to concatenate

In [28]:
# Compute metrics per model AND per split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr


def compute_metrics(df: pd.DataFrame) -> dict:
    y_true = df['y_true'].astype(float)
    y_pred = df['y_pred'].astype(float)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    if y_true.nunique() > 1 and y_pred.nunique() > 1:
        pearson = pearsonr(y_true, y_pred)[0]
        spearman = spearmanr(y_true, y_pred)[0]
    else:
        pearson = np.nan
        spearman = np.nan
    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2,
        'Pearson': pearson,
        'Spearman': spearman,
    }

metrics_rows = []
for (name, split), g in all_preds.groupby(['model','Split']):
    row = {'model': name, 'Split': split}
    row.update(compute_metrics(g))
    metrics_rows.append(row)

metrics_df = pd.DataFrame(metrics_rows).sort_values(by=['Split','MAE','RMSE'])
metrics_df.reset_index(drop=True, inplace=True)
metrics_df

/var/folders/_0/j8xw5t8j6p3dg7_fq82t36gr0000gp/T/ipykernel_24125/163650798.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (name, split), g in all_preds.groupby(['model','Split']):


,model,Split,MAE,MSE,RMSE,R2,Pearson,Spearman
0,GBM_davis,Test,0.648944,8.725102e-01,0.934083,0.501232,0.739670,0.648687
1,XGBoost_davis,Test,0.834103,1.614732e+00,1.270721,0.076942,0.413499,0.737328
2,Random_Forest_davis,Test,0.934707,2.479980e+00,1.574795,-0.417675,0.132560,0.264675
3,Linear_Regression_davis,Test,0.991010,1.727687e+00,1.314415,0.012372,0.698735,0.511698
4,SVR_davis,Test,1.013706,1.864697e+00,1.365539,-0.065950,0.674840,0.531843
5,MLP_davis,Test,4.494602,2.183891e+01,4.673212,-11.484165,0.463937,0.390824
6,XGBoost_davis,Train,0.000253,1.288653e-07,0.000359,1.000000,1.000000,0.923264
7,GBM_davis,Train,0.003345,1.656483e-05,0.004070,0.999984,0.999994,0.923256
8,Random_Forest_davis,Train,0.645185,1.341708e+00,1.158321,-0.330994,0.327631,0.389897
9,Linear_Regression_davis,Train,0.670518,9.009299e-01,0.949173,0.106265,0.911600,0.873643


In [29]:
metrics_df.to_csv(results_dir / 'model_evaluation_metrics_by_split_Davis.csv', index=False)

PosixPath('/Users/cheng.wang/Documents/GitHub/llm-drug-agent/Evaluation_Metrics/figures/pred_vs_true_scatter_by_model_hue_split_Davis.png')

In [30]:
# Scatter plots: Predicted vs True per model, colored by Split (one subplot per model)
fig_dir = results_dir.parent / 'Evaluation_Metrics' / 'figures'
fig_dir.mkdir(parents=True, exist_ok=True)

# Determine splits present and build a safe palette
splits_present = [s for s in pd.Series(all_preds['Split']).astype(str).dropna().unique().tolist()]
palette_base = {
    'Train': '#1f77b4',
    'Validation': '#ff7f0e',
    'Test': '#2ca02c',
    'Unknown': '#7f7f7f',
}
palette = {k: palette_base.get(k, '#7f7f7f') for k in splits_present}

# Facet by Model; color by Split
g = sns.FacetGrid(all_preds, col="model", sharex=True, sharey=True, height=3.2, margin_titles=True, hue="Split", palette=palette)
g.map_dataframe(sns.scatterplot, x='y_true', y='y_pred', s=14, alpha=0.65)

# Add y=x line to each facet
min_v = np.nanmin(all_preds[['y_true','y_pred']].values)
max_v = np.nanmax(all_preds[['y_true','y_pred']].values)
for ax in g.axes.flatten():
    ax.plot([min_v, max_v], [min_v, max_v], ls='--', c='red', lw=1)
    ax.set_xlabel('True Affinity')
    ax.set_ylabel('Predicted Affinity')

# Add legend once
g.add_legend(title='Split')

g.fig.suptitle('Predicted vs True Affinity by Model (colored by Split)', y=1.02)
plt.tight_layout()
scatter_path = fig_dir / 'pred_vs_true_scatter_by_model_hue_split_Davis.png'
plt.savefig(scatter_path, dpi=300, bbox_inches='tight')
scatter_path

PosixPath('/Users/cheng.wang/Documents/GitHub/llm-drug-agent/Evaluation_Metrics/figures/residuals_boxplot_by_split_Davis.png')

In [31]:
# Residual box plot per model and split
all_preds['residual'] = all_preds['y_pred'] - all_preds['y_true']

plt.figure(figsize=(10, 4))
sns.boxplot(data=all_preds, x='model', y='residual', hue='Split')
plt.axhline(0, ls='--', c='red', lw=1)
plt.xticks(rotation=45, ha='right')
plt.title('Residuals per Model by Split (y_pred - y_true)')
plt.legend(title='Split', bbox_to_anchor=(1.02, 1), loc='upper left')
box_path = fig_dir / 'residuals_boxplot_by_split_Davis.png'
plt.tight_layout()
plt.savefig(box_path, dpi=300)
box_path

## Metrics summary
The table below summarizes error and correlation metrics per model. Lower MAE/RMSE and higher R2/Correlations are better.

In [32]:
# Show metrics
metrics_df

,model,Split,MAE,MSE,RMSE,R2,Pearson,Spearman
0,GBM_davis,Test,0.648944,8.725102e-01,0.934083,0.501232,0.739670,0.648687
1,XGBoost_davis,Test,0.834103,1.614732e+00,1.270721,0.076942,0.413499,0.737328
2,Random_Forest_davis,Test,0.934707,2.479980e+00,1.574795,-0.417675,0.132560,0.264675
3,Linear_Regression_davis,Test,0.991010,1.727687e+00,1.314415,0.012372,0.698735,0.511698
4,SVR_davis,Test,1.013706,1.864697e+00,1.365539,-0.065950,0.674840,0.531843
5,MLP_davis,Test,4.494602,2.183891e+01,4.673212,-11.484165,0.463937,0.390824
6,XGBoost_davis,Train,0.000253,1.288653e-07,0.000359,1.000000,1.000000,0.923264
7,GBM_davis,Train,0.003345,1.656483e-05,0.004070,0.999984,0.999994,0.923256
8,Random_Forest_davis,Train,0.645185,1.341708e+00,1.158321,-0.330994,0.327631,0.389897
9,Linear_Regression_davis,Train,0.670518,9.009299e-01,0.949173,0.106265,0.911600,0.873643
